# Advanced ensembles

This week we are going to review more advanced ensembles in Sklearn and... XGBoost! 

We will focus on the following algorithms: 

* AdaBoost
* GradientBoost (Sklearn)
* XGBoost (XGBoost library)

Our example data this week will be the Local Weather Archive in the Town of Cary, North Carolina. The dataset contains several input features about weather parameters, which we can use to predict whether it rained or not during a given week (binary classification).

In [ ]:
# Let's start by installing our little baby:
!pip install xgboost

In [ ]:
# And importing some stuff from SKlearn to run too:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import KNNImputer

from sklearn.model_selection import cross_validate

import io
from itertools import product

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import requests

In [ ]:
# Now as usual, we will pick the open-data from a URL and create our Pandas DataFrame:
data_url = 'https://data.townofcary.org/api/v2/catalog/datasets/rdu-weather-history/exports/csv'

data_stream=requests.get(data_url).content
df=pd.read_csv(io.StringIO(data_stream.decode('utf-8')), sep=';', engine='python')
print('Rows in data:', len(df))
print('Columns in data:', len(df.columns))
list(df.columns)

In [ ]:
columns_to_delete = [
    'fastest5secwinddir',
    'fastest5secwindspeed', # always null
    'date', # we won't use date
    'precipitation', # we are going to classify a binary column called "Rain", if precipitation>0, then Rain is True always...
    'drizzle', # if it drizzled, it rained...
    'snow', # if it snowed, it rained too..
    'blowingsnow',
    'hail', # and same with hail
]

df.drop(columns_to_delete, axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Columns from 7 onwards are all boolean:
boolean_columns = df.columns[7:]
print(list(boolean_columns))

def make_dummy(value):
    if(value == 'Present'):
        return str(1)
    else:
        return str(0)
    
for b in boolean_columns:
    df[b] = df[b].apply(make_dummy)

In [ ]:
# Here I am just putting the 'rain' feature at the end of the DataFrame, nothing else (I like to have the target at the end):

# IF YOU WANT TO PREDICT SOMETHING ELSE LIKE: mist, fog, fogheavy, ice, ETC. JUST CHANGE THIS!
target_feature_name = 'rain'
input_features = [c for c in df.columns if c != target_feature_name]

target_feature = df[target_feature_name]
df = df[input_features]
df['rain'] = target_feature

In [ ]:
# Finally, here we will impute the NaN values (not-a-number), using a KNN imputer: Setting the values of the NaN data points
# as the average of their 5 nearest neighours.

imputer = KNNImputer(missing_values=np.NaN, n_neighbors=5, weights='distance')
df[input_features] = imputer.fit_transform(df[input_features])

In [ ]:
df

Alright, data is ready to rock!!

#### Let's start with Sklearn:

RandomForest, Bagging, AdaBoost and GradientBoost :)

In [ ]:
NUM_ESTIMATORS = 50 # 50 classifiers in each ensemble. FEEL FREE TO CHANGE THIS!

X = df[input_features]
y = df[target_feature_name].apply(int)

In [ ]:
# We are going to measure several parameters this time, not just accuracy. YOU CAN ADD OR REMOVE AS YOU LIKE
metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

def print_results(scores):
    for key in scores:
        if key.startswith('test_'):
            print(f'{key}:\t{round(scores[key].mean(), 4)}')

In [ ]:
baggingDT = BaggingClassifier(
    base_estimator=None, # When this is NULL, Bagging will use DecisionTrees as the base estimator
    n_estimators=NUM_ESTIMATORS,
    max_samples=0.30,    # A maximum of 30% of the data will be used in each DecisionTree, FEEL FREE TO CHANGE THIS
    n_jobs=-1,
)
print('Doing, ', baggingDT)
bagging_score = cross_validate(
    baggingDT, X, y, cv=5, scoring=metrics
)
print(baggingDT, 'done')
print_results(bagging_score)

In [ ]:
randomForest = RandomForestClassifier(
    n_estimators=NUM_ESTIMATORS,
    n_jobs=-1,
)
print('Doing, ', randomForest)
rf_score = cross_validate(
    randomForest, X, y, cv=5, scoring=metrics
)
print(randomForest, 'done')
print_results(rf_score)

In [ ]:
adaBoost = AdaBoostClassifier(
    n_estimators=NUM_ESTIMATORS,
    learning_rate=1,      # FEEL FREE TO CHANGE THIS TO TEST FOR BETTER RESULTS
    algorithm='SAMME',    # SAMME works well for discrete outputs (SAMME.R is for regression remember). We don't have M1 for binary in Sklearn unfortunately
    # n_jobs=-1, --> Because of the way it works (chaining/cascading classifiers), it cannot be parallellised!!
)
print('Doing, ', adaBoost)
ab_score = cross_validate(
    adaBoost, X, y, cv=5, scoring=metrics
)
print(adaBoost, 'done')
print_results(ab_score)

In [ ]:
gradBoost = GradientBoostingClassifier(
    n_estimators=NUM_ESTIMATORS,
    learning_rate=0.1,    # FEEL FREE TO CHANGE THIS TO TEST FOR BETTER RESULTS
    subsample=1,        # Each new classifier will be trained on 100% of the data (no bagging). If it's less than 1 (100%), we will be doing a STOCHASTIC gradient boosting
    # n_jobs=-1, --> Because of the way it works (chaining/cascading classifiers), it cannot be parallellised!!
)
print('Doing, ', gradBoost)
gb_score = cross_validate(
    gradBoost, X, y, cv=5, scoring=metrics
)
print(gradBoost, 'done')
print_results(gb_score)

In [ ]:
score_overview = {
    'bagging': bagging_score,
    'randomforest': rf_score,
    'adaboost': ab_score,
    'gradient_boost': gb_score,
}

scores_data = []
for algo in score_overview:
    for metric in score_overview[algo]:
        row = [algo]
        row.append(metric)
        row.append(score_overview[algo][metric].mean())
        scores_data.append(row)

In [ ]:
scores = pd.DataFrame(scores_data)
scores.columns = ['algorithm', 'metric', 'score']

In [ ]:
scores

In [ ]:
plt.rcParams['figure.figsize'] = (15,7)
sns.barplot(data=scores, x='metric', y='score', hue='algorithm', saturation=0.6, palette='Blues',)

#### Summary:
(As the results were when I ran this Notebook - you might get slightly different ones as cross-validation is shuffling the data) 

**fit_time** and **score_time**
While AdaBoost and GradientBoost are slower when fitting the training data, they are much faster when making predictions (once the model is trained, to output a class for new input data). Particularly GradientBoost is just a small fraction of the rest. So once the model is trained, it will be much faster in a production environment.

**Accuracy**
Accuracies are quite similar for all 4 ensemble classifiers - however GradientBoost peaks a little bit over the rest.

**Precision** and **Recall**
Again more or less the same in all of them. AdaBoost appears to have sacrificied a little bit of precision to maximise recall in this instance. Bagging and RandomForest are the opposite than AdaBoost. GradientBoost seems to be balancing the two.

**F1-score** and **ROC AUC**
AdaBoost and GradientBoost are slightly ahead, but not much.

**In short**
All four algorithhms perform more or less the same with this data - however we can see very different training and testing times. They all have a lot of parameters that we can tune to make them much better than they are in this Notebook (depth of Decision Trees, maximum number of leaf nodes, minimum number of samples represented in each node, **learning rate** in AdaBoost and GradientBoost... etc.

#### Use the cell below to play with Gradient Boosting and see if you can achieve better results
Change the parameters as you like and see how the performance changes.

Check the documentation [here to better understand the parameters](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier) that Gradient Boost uses. As with RandomForest, many ot them relate to the underlying Decition Trees:

In [ ]:
gradBoost = GradientBoostingClassifier(
    n_estimators=10,     # We will use again 100 DecisionTreeClassifiers, FEEL FREE TO CHANGE THIS
    learning_rate=0.5,    # FEEL FREE TO CHANGE THIS TO TEST FOR BETTER RESULTS
    subsample=1,        # Each new classifier will be trained on 100% of the data (no bagging). If it's less than 1 (100%), we will be doing a STOCHASTIC gradient boosting
    loss='deviance', 
    criterion='friedman_mse', 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_depth=3, 
    min_impurity_decrease=0.0, 
    min_impurity_split=None, 
    init=None, 
    random_state=None, 
    max_features=None, 
    verbose=0, 
    max_leaf_nodes=None, 
    warm_start=False, 
    validation_fraction=0.1, 
    n_iter_no_change=None, 
    tol=0.0001, 
    ccp_alpha=0.0
)
print('Starting...')
gb_score = cross_validate(
    gradBoost, X, y, cv=5, scoring=metrics
)
print('Done')
print_results(gb_score)

### Overfitting? Let's experiment

As a test, we will sub-sample our original data to make the two classes perfectly balanced and we will re-train our models using **just 2 features** (you can select which 2 down below). And then, we will plot the decision boundaries of the classifiers.

In the plots below the blue areas are places where the classifier predicted that it will rain - and the red ones it is where it decided that it will not rain.

The classifiers with many small areas are likely to be more overfitted (creating a blue areas for every single rain sample), while the ones with large and well defined blue areas are likely to generalise better.

**There are not conclusive results**, just for me to show how we can plot this with our current data. Don't even think that we can extrapolate this and state: AdaBoost overfits less than RandomForest, for example.

In [ ]:
# Plotting decision regions:
# CHOOSE ANY TWO INPUT FEATURES TO SEE HOW THE CLASSIFIERS DECIDED THE DECISION BOUNDARIES BETWEEN THEM!
x_axis = 'temperaturemin'
y_axis = 'temperaturemax'



# Sub-sampling
sample_data = df[df['rain'] == '0'].sample(len(df[df['rain'] == '1']))
sample_data = sample_data.append(df[df['rain'] == '1']).sample(frac=1)

# I will fit the classifiers using all of the data, so later we will plot the decision boundaries for the entire dataset:
classifiers = [baggingDT, randomForest, adaBoost, gradBoost]
for c in classifiers:
    c.fit(sample_data[[x_axis, y_axis]], sample_data[target_feature_name])

In [ ]:
x_min, x_max = X[x_axis].min() - 1, X[x_axis].max() + 1
y_min, y_max = X[y_axis].min() - 1, X[y_axis].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

f, axarr = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(25, 15))
plt.set_cmap('coolwarm')

for idx, clf, tt in zip(product([0, 1], [0, 1]),
                        classifiers,
                        ['Bagging', 'Random Forest', 'AdaBoost', 'Gradient Boost']):

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    axarr[idx[0], idx[1]].contourf(xx, yy, Z, alpha=0.2)
    axarr[idx[0], idx[1]].scatter(sample_data[x_axis], sample_data[y_axis], c=sample_data[target_feature_name], s=8)
    axarr[idx[0], idx[1]].set_title(tt)

plt.show()

In [ ]:
# XGBoost

import xgboost as xgb

# Here is where you will need to get your computer science skills and get the dependencies installed.
# In a Mac, I just had to `brew install libomp`. It might be more complicated in other systems.

# As XGBoost becomes more and more standard, we expect the installation to become easier.

# NOTE: THIS PART IS COMPLETELY OPTIONAL, NOT PART OF THE COURSE. You don't need to know the XGBoost library 
#(just the theory of the slides)

In [ ]:
# DMatrix is like Pandas DataFrame. We need to create it in order to train an XGBoost model with it:
dmatrix = xgb.DMatrix(X, label=y)

'''
    Below I add some code cells designed to make you understand how DMatrices are used, even if you didn't manage to install 
    XGBoost in your machine (but maybe in the future in some work computer it'll be already installed so you can have some 
    knowledge to get started)
'''

In [ ]:
dmatrix.feature_names

In [ ]:
print('Num rows:', dmatrix.num_row(), 'Num cols:', dmatrix.num_col())

In [ ]:
# Let's start with XGBoost itself. First, we define a dictionary with the parameters of our model:
model_parameters = {
#     'n_estimators': 50, -> We can either set n_estimators like in Sklearn, or nb_rounds (I have it below)
    'max_depth': 2,
    'eta': 1.0,   # This is the learning rate! (As in Sklearn's GradientBoost)
    'objective': 'binary:hinge', # There are many objective functions, depending on the type of the target variable and the metrics we want to evaluate it against 
    'eval_metric': ['auc'], # This part is very different to Sklearn. You will need to get familiar with many other metrics
#     'lambda': .1,
    'nthread': 4, # my computer has 4 cores - change to maximise the speed according to your computer cores
}

In [ ]:
nb_rounds = 50 #Number of boosting rounds!
xgb_model = xgb.cv(
    model_parameters, 
    dmatrix,
    nb_rounds,
    nfold=5,
)

In [ ]:
xgb_model.describe()

In [ ]:
# We will train a XGBoost model using the train method (much like the fit method in Sklearn)
model = xgb.train(model_parameters, dmatrix)

In [ ]:
# And XGBoost has a function to see the feature importances:
xgb.plot_importance(model)

#### And that's the last thing for today...

Do you remember that we could use DecisionTrees and RandomForest to do some feature selection? 

Well all of the ensembles based on DecisionTrees* can do this too!! (as we just saw with XGBoost)

(Apart from Bagging, which is not implemented in Sklearn)

In [ ]:
for c in classifiers:
    c.fit(X, y)
    if hasattr(c, 'feature_importances_'):
        print(c)
        imp_feat = sorted(zip(input_features, c.feature_importances_), key=lambda x: -x[1])[:5]
        for f, i in imp_feat:
            print(f, '\t', round(i, 4))

### They all seem to agree, more or less :)

# Learning Exercises:

* The learning exercises are already laid down in the code comments throughout the notebook: Can you tune these ensembles to improve the performances I achieved? Tweak the different attributes of each method to maximise it. Maybe even run some Grid/Random optimiser
* In the decision boundaries part: Can you reuse this code and show the decision boundaries for other classifiers, trained with 2 features? (e.g. SVM, KNN, DecisionTree, NaiveBayes, etc, etc). You can learn a lot about how all of the classifiers, and their variations (kernels, parameters), work. Can be a great reviewing exercise
* For those of you who managed to get XGBoost installed: can you match the ROC AUC that we got in the other ensembles? (if you do, let me know)
    